In [ ]:
#create new database to contain Boston crime data
import psycopg2
conn = psycopg2.connect(dbname="postgres", user="postgres", password="practiceserver")
cur = conn.cursor()
conn.autocommit = True
cur.execute("CREATE DATABASE boston_crime_db")
conn.autocommit = False
conn.close()

In [13]:
#create a crimes schema
conn = psycopg2.connect(dbname="boston_crime_db", user="postgres", password="practiceserver") 
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes")
conn.commit()
conn.close()

In [19]:
#print the headers and first row from the csv file
import csv
with open("boston_crimes.csv") as file:
    boston_read = csv.reader(file)
    headers = next(boston_read)
    first_row = next(boston_read)
    print(headers,"\n", first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long'] 
 ['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [35]:
#create a function returning a set of column values
def get_column(csv_file, col_index):
    col_contents = set()
    import csv
    with open(csv_file) as file:
        next(file)
        read_file = csv.reader(file)
        for row in read_file:
            col_content = row[col_index]
            col_contents.add(col_content)
    return col_contents        

In [42]:
#find the number of unique values in each column
for column in range(len(headers)):
    column_contents = get_column("boston_crimes.csv", column)
    print(headers[column], len(column_contents), sep=" -- ")

incident_number -- 298329
offense_code -- 219
description -- 239
date -- 1177
day_of_the_week -- 7
lat -- 18177
long -- 18177


In [53]:
#description column would contain the most characters, find max length of 58
description_col = get_column("boston_crimes.csv", 2)

longest_char = 0
char = "blank"
for value in description_col:
    value_length = len(value)
    if value_length > longest_char:
        longest_char = value_length
        char = value
print(longest_char," -> ",char)    

58  ->  RECOVERED - MV RECOVERED IN BOSTON (STOLEN OUTSIDE BOSTON)


In [65]:
#creating a new table in the database
conn = psycopg2.connect(dbname="boston_crime_db", user="postgres", password="practiceserver") 
cur = conn.cursor()

print(headers,"\n",first_row)

#enumerate days of the week as there are only 7 fixed values
cur.execute("""
CREATE TYPE day AS ENUM('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday')
""")

#use varchar 100 for description column to give leeway for max of 58 characters
cur.execute("""
CREATE TABLE crimes.boston_crimes (
incident_number integer PRIMARY KEY,
offense_code integer,
description varchar(100),
date date,
day_of_the_week day,
lat decimal,
long decimal)
""")

conn.commit()

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long'] 
 ['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [71]:
#load the data from the csv into the table
with open("boston_crimes.csv") as file:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER",file)
conn.commit()

In [76]:
#verify
cur.execute("SELECT* FROM crimes.boston_crimes LIMIT 3")
cur.fetchall()

[(1,
  619,
  'LARCENY ALL OTHERS',
  datetime.date(2018, 9, 2),
  'Sunday',
  Decimal('42.35779134'),
  Decimal('-71.13937053')),
 (2,
  1402,
  'VANDALISM',
  datetime.date(2018, 8, 21),
  'Tuesday',
  Decimal('42.30682138'),
  Decimal('-71.06030035')),
 (3,
  3410,
  'TOWED MOTOR VEHICLE',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.34658879'),
  Decimal('-71.07242943'))]

In [80]:
#revoke privileges from the public group on the public schema
#revoke privileges on the new database from the public group
cur.execute("REVOKE ALL ON SCHEMA public FROM public")
cur.execute("REVOKE ALL ON DATABASE boston_crime_db FROM public")

In [87]:
#create a read and read/write group
cur.execute("CREATE GROUP read NOLOGIN")
cur.execute("CREATE GROUP readwrite NOLOGIN")

#give both the ability to connect to the database
cur.execute("GRANT CONNECT ON DATABASE boston_crime_db TO read")
cur.execute("GRANT CONNECT ON DATABASE boston_crime_db TO readwrite")

#give both access to the crimes schema
cur.execute("GRANT USAGE ON SCHEMA crimes TO read")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite")

#give the read group the ability to use select statements
#give the read/write group the ability to also insert, delete, and update tables
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO read")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite")

conn.commit()

In [99]:
#verify privileges
cur.execute("""
SELECT grantor, grantee, privilege_type 
FROM information_schema.table_privileges 
WHERE grantee ='read' OR grantee ='readwrite'
""")
cur.fetchall()

[('postgres', 'read', 'SELECT'),
 ('postgres', 'readwrite', 'INSERT'),
 ('postgres', 'readwrite', 'SELECT'),
 ('postgres', 'readwrite', 'UPDATE'),
 ('postgres', 'readwrite', 'DELETE')]

In [100]:
#create sample users for both groups
cur.execute("CREATE USER example_reader WITH PASSWORD 'examplepassword1'")
cur.execute("CREATE USER example_readwriter WITH PASSWORD 'examplepassword2'")

#assign the sample users to respective groups
cur.execute("GRANT read TO example_reader")
cur.execute("GRANT readwrite TO example_readwriter")
conn.commit()